# Propogate Errors

## THEANO BACKEND ONLY

## This notebook takes you through the steps of how to propogate errors for through the neural network model

## required packages:
- numpy
- h5py
- Keras 
  - must be using the same version used to train the model
- Theano 
  - backend for Keras, Theano has a Jacobian function that allows for the calculation of the partial derivatives for the model, which is necessary for the error propogation. Your model must have also been trained using Theano
- Matplotlib and seaborn are only necessary to make plots of predictions vs. ASPCAP Dr12 labels

## required data files:
- model file
  - can be created in $4\_Train\_Model\_Keras\_2.ipynb$ or $4\_Train\_Model\_Keras\_2.ipynb$
- test set file
  - can be created in $3\_Preprocessing\_of\_Test\_Data.ipynb$ or downloaded  in $1\_Download\_Data.ipynb$
- mean_and_std.npy
  - can be created in $3\_Preprocessing\_of\_Test\_Data.ipynb$ or downloaded in $1\_Download\_Data.ipynb$
- apStar_combined_main.h5
  - can be downloaded in $1\_Download\_Data.ipynb$

In [1]:
import os
os.environ["KERAS_BACKEND"] = 'theano'

In [2]:
import numpy as np
from keras.models import load_model
import h5py
import theano
import theano.tensor as T

Using Theano backend.


## Obtain data for normalizing labels

In [3]:
mean_and_std = np.load('mean_and_std.npy')
mean_labels = mean_and_std[0]
std_labels = mean_and_std[1]
num_labels = mean_and_std.shape[1]

## Define function to denormalize labels

In [4]:
def denormalize(lb_norm):
    return ((lb_norm*std_labels)+mean_labels)

## Obtain test set

In [5]:
savename = 'high_snr_test_data.h5'

In [6]:
with h5py.File(savename,"r") as F:
    test_1_spectra = F["spectra"][:]
    test_1_spectra = test_1_spectra.reshape((test_1_spectra.shape[0],test_1_spectra.shape[1],1))
    test_1_error_spectra = F["error_spectra"][:]
    test_1_labels = np.column_stack((F["TEFF"][:],F["LOGG"][:],F["FE_H"][:]))
print('Test set includes '+str(len(test_1_spectra))+' combined spectra.')

Test set includes 2780 combined spectra.


# Load Model

In [7]:
model_name = 'Model_Theano_Keras_1.h5'
model = load_model(model_name)
print(model_name+' loaded.')

Model_Theano_Keras_1.h5 loaded.


# Predict on test set

In [8]:
test_1_pred = denormalize(model.predict(test_1_spectra))

# Load entire APOGEE dataset
This is necessary to obtain the an accurate assessment of the scatter between the model predictions and apogee labels

In [9]:
savename = '/data/stars/teaghan/apStar_combined_main.h5'
with h5py.File(savename,"r") as F:
    all_apogee_spectra = F["spectrum"][:]
    all_apogee_labels = np.column_stack((F["TEFF"][:],F["LOGG"][:],F["FE_H"][:]))

# Normalize spectra
same method as was used in  $2\_Preprocessing\_of\_Training\_Data.ipynb$ and $3\_Preprocessing\_of\_Test\_Data.ipynb$

In [10]:
# Define edges of detectors
blue_chip_begin = 322
blue_chip_end = 3242
green_chip_begin = 3648
green_chip_end = 6048   
red_chip_begin = 6412
red_chip_end = 8306 

# Separate spectra into chips

blue_sp = all_apogee_spectra[:,blue_chip_begin:blue_chip_end]
green_sp = all_apogee_spectra[:,green_chip_begin:green_chip_end]
red_sp = all_apogee_spectra[:,red_chip_begin:red_chip_end]

#Normalize spectra by chips

blue_sp_med = np.median(blue_sp, axis=1)
green_sp_med = np.median(green_sp, axis=1)
red_sp_med = np.median(red_sp, axis=1)

blue_sp = (blue_sp.T/blue_sp_med).T
green_sp = (green_sp.T/green_sp_med).T
red_sp = (red_sp.T/red_sp_med).T  

# Recombine spectra

all_apogee_spectra = np.column_stack((blue_sp,green_sp,red_sp))

print('Large APOGEE dataset now contains '+str(all_apogee_spectra.shape[0])+' spectra, each with '+str(all_apogee_spectra.shape[1])+' wavelength bins')

Large APOGEE dataset now contains 147590 spectra, each with 7214 wavelength bins


reshape spectra:

In [11]:
all_apogee_spectra = all_apogee_spectra.reshape((all_apogee_spectra.shape[0],all_apogee_spectra.shape[1],1))

### make predictions on large APOGEE dataset

In [12]:
all_apogee_pred = denormalize(model.predict(all_apogee_spectra))

calculate residuals:

In [13]:
all_apogee_diffs = all_apogee_pred-all_apogee_labels

## separate residuals into different bins
This is necessary to undestand how the scatter in different ranges of the label-space differs so that the appropriate scatter values are used when including the scatter in the error propogation 

In [14]:
diff_t1 = all_apogee_diffs[np.where((all_apogee_labels[:,0]<4000)&(all_apogee_labels[:,0]>10))[0]]
diff_t2 = all_apogee_diffs[np.where((all_apogee_labels[:,0]<4500)&(all_apogee_labels[:,0]>4000))[0]]
diff_t3 = all_apogee_diffs[np.where((all_apogee_labels[:,0]<4750)&(all_apogee_labels[:,0]>4500))[0]]
diff_t4 = all_apogee_diffs[np.where((all_apogee_labels[:,0]<5250)&(all_apogee_labels[:,0]>4750))[0]]
diff_t5 = all_apogee_diffs[np.where((all_apogee_labels[:,0]>5250))[0]]

diff_l1 = all_apogee_diffs[np.where((all_apogee_labels[:,1]<0.5)&(all_apogee_labels[:,1]>-10.))[0]]
diff_l2 = all_apogee_diffs[np.where((all_apogee_labels[:,1]<1.5)&(all_apogee_labels[:,1]>0.5))[0]]
diff_l3 = all_apogee_diffs[np.where((all_apogee_labels[:,1]<2.5)&(all_apogee_labels[:,1]>1.5))[0]]
diff_l4 = all_apogee_diffs[np.where((all_apogee_labels[:,1]<3.5)&(all_apogee_labels[:,1]>2.5))[0]]
diff_l5 = all_apogee_diffs[np.where((all_apogee_labels[:,1]>3.5))[0]]

diff_f1 = all_apogee_diffs[np.where((all_apogee_labels[:,2]<-1.3)&(all_apogee_labels[:,2]>-10.))[0]]
diff_f2 = all_apogee_diffs[np.where((all_apogee_labels[:,2]<-0.9)&(all_apogee_labels[:,2]>-1.3))[0]]
diff_f3 = all_apogee_diffs[np.where((all_apogee_labels[:,2]<-0.5)&(all_apogee_labels[:,2]>-0.9))[0]]
diff_f4 = all_apogee_diffs[np.where((all_apogee_labels[:,2]<-0.3)&(all_apogee_labels[:,2]>-0.5))[0]]
diff_f5 = all_apogee_diffs[np.where((all_apogee_labels[:,2]>-0.3))[0]]

### obtain a random sample of residuals from each bin
Each sample has to be equal in size for proper statistical analysis

In [15]:
np.random.shuffle(diff_t1)
diff_t1 = diff_t1[0:1500]
np.random.shuffle(diff_t2)
diff_t2 = diff_t2[0:1500]
np.random.shuffle(diff_t3)
diff_t3 = diff_t3[0:1500]
np.random.shuffle(diff_t4)
diff_t4 = diff_t4[0:1500]
np.random.shuffle(diff_t5)
diff_t5 = diff_t5[0:1500]

np.random.shuffle(diff_l1)
diff_l1 = diff_l1[0:1500]
np.random.shuffle(diff_l2)
diff_l2 = diff_l2[0:1500]
np.random.shuffle(diff_l3)
diff_l3 = diff_l3[0:1500]
np.random.shuffle(diff_l4)
diff_l4 = diff_l4[0:1500]
np.random.shuffle(diff_l5)
diff_l5 = diff_l5[0:1500]

np.random.shuffle(diff_f1)
diff_f1 = diff_f1[0:1500]
np.random.shuffle(diff_f2)
diff_f2 = diff_f2[0:1500]
np.random.shuffle(diff_f3)
diff_f3 = diff_f3[0:1500]
np.random.shuffle(diff_f4)
diff_f4 = diff_f4[0:1500]
np.random.shuffle(diff_f5)
diff_f5 = diff_f5[0:1500]

### calculate scatter in different regions, $\delta_{js}$

In [16]:
std_diff_t1 = np.std(diff_t1, axis=0)[0]
std_diff_t2 = np.std(diff_t2, axis=0)[0]
std_diff_t3 = np.std(diff_t3, axis=0)[0]
std_diff_t4 = np.std(diff_t4, axis=0)[0]
std_diff_t5 = np.std(diff_t5, axis=0)[0]

std_diff_l1 = np.std(diff_l1, axis=0)[1]
std_diff_l2 = np.std(diff_l2, axis=0)[1]
std_diff_l3 = np.std(diff_l3, axis=0)[1]
std_diff_l4 = np.std(diff_l4, axis=0)[1]
std_diff_l5 = np.std(diff_l5, axis=0)[1]

std_diff_f1 = np.std(diff_f1, axis=0)[2]
std_diff_f2 = np.std(diff_f2, axis=0)[2]
std_diff_f3 = np.std(diff_f3, axis=0)[2]
std_diff_f4 = np.std(diff_f4, axis=0)[2]
std_diff_f5 = np.std(diff_f5, axis=0)[2]

# Create a Theano Function that returns the Jacobian matrix
The jacobian matrix is a matrix of the first order derivatives of the outputs with respect to the input. In our case, this will be a 3-dimensional matrix with dimensions: (num_labels, num_test_spectra, num_flux_values).

Each spectrum will therefore have 3 vectors the length of the spectrum: one vector for each of the first order derivatives of the output labels with respect to each flux value (wavelength bin)

In [17]:
jac = theano.gradient.jacobian(denormalize(model.layers[-1].output).flatten(),wrt=model.layers[0].input)
compute_jac = theano.function([model.layers[0].input],[jac],allow_input_downcast=True)

### Calculate Jacobian Matrix
\begin{equation}
\frac{\partial h_{j}(\textbf{x},\textbf{W})}{\partial \textbf{x}} =  (\frac{\partial h_{j}(\textbf{x},\textbf{W})}{\partial x_{1}},...,\frac{\partial h_{j}(\textbf{x},\textbf{W})}{\partial x_{n}})
\end{equation} 

j = 1,...,3

n = 1,...,7214

In [18]:
jacobian_matrix = np.zeros((3,1,7214))
for i in range(len(test_1_spectra)):
    a = compute_jac(test_1_spectra[i:i+1])[0].reshape(3,1,7214)
    if i ==0:
        jacobian_matrix = a
    else:
        jacobian_matrix = np.column_stack((jacobian_matrix,a))

Mask extremely high values in the error spectra and nan values in jacobian.

The high values in the error spectrum are associated - for the most part - with zero-values in the APOGEE spectra. Since these zero values are essentially ignored in the model (due to RELU-activation and maxpooling layers) they do not effect the output labels and therefore, the flux errors associated with these zero-values give an innaccurate assessment of the prediction errors. If you were to include these error fluxes, you would have massive uncertainties in some of the stars' output labels

In [19]:
test_1_error_spectra[test_1_error_spectra > 5] = 0
jacobian_matrix = np.nan_to_num(jacobian_matrix)

### Combine first order derivatives with error spectra

\begin{equation}
\delta_{j\textbf{x}} = \frac{\partial h_{j}(\textbf{x},\textbf{W})}{\partial \textbf{x}} \cdot \Delta \textbf{x}
\end{equation}

In [20]:
test_1_pred_errors = np.array((np.einsum('ij,ij->i', np.square(jacobian_matrix[0]), np.square(test_1_error_spectra)),np.einsum('ij,ij->i', np.square(jacobian_matrix[1]), np.square(test_1_error_spectra)),np.einsum('ij,ij->i', np.square(jacobian_matrix[2]), np.square(test_1_error_spectra)))).T

In [21]:
# label names
label_names = ['Teff  ','log(g)','[Fe/H]']
units = ['K','cgs','dex']

In [22]:
mean_err_sp = np.mean(np.sqrt(test_1_pred_errors), axis=0)
print('Mean error due to error spectrum: \n')
for i, err in enumerate(mean_err_sp):
      print(label_names[i]+':  '+"{0:.3f}".format(err)+' '+units[i])

Mean error due to error spectrum: 

Teff  :  13.513 K
log(g):  0.033 cgs
[Fe/H]:  0.012 dex


### Combine the previous errors with scatter from the corresponding bins

\begin{equation}
\Delta h_{j} = \sqrt{\delta_{j\textbf{x}}^{2}  + \delta_{js}^{2}}
\end{equation}

In [23]:
test_1_total_err = np.zeros((len(test_1_pred),3))
for i, label in enumerate(test_1_pred):
    std_diff_temp = np.zeros((1,3))
    if (label[0]<4000) & (label[0]>10):
        std_diff_temp[0,0]=std_diff_t1
    elif (label[0]<4500) & (label[0]>4000):
        std_diff_temp[0,0]=std_diff_t2
    elif (label[0]<4750) & (label[0]>4500):
        std_diff_temp[0,0]=std_diff_t3
    elif (label[0]<5250) & (label[0]>4750):
        std_diff_temp[0,0]=std_diff_t4
    elif (label[0]<10000) & (label[0]>5250):
        std_diff_temp[0,0]=std_diff_t5
        
    if (label[1]<0.5) & (label[0]>-10):
        std_diff_temp[0,1]=std_diff_l1
    elif (label[1]<1.5) & (label[0]>0.5):
        std_diff_temp[0,1]=std_diff_l2
    elif (label[1]<2.5) & (label[0]>1.5):
        std_diff_temp[0,1]=std_diff_l3
    elif (label[1]<3.5) & (label[0]>2.5):
        std_diff_temp[0,1]=std_diff_l4
    elif (label[1]<100) & (label[0]>3.5):
        std_diff_temp[0,1]=std_diff_l5
    
    if (label[2]<-1.3) & (label[0]>-10):
        std_diff_temp[0,2]=std_diff_f1
    elif (label[2]<-0.9) & (label[0]>-1.3):
        std_diff_temp[0,2]=std_diff_f2
    elif (label[2]<-0.5) & (label[0]>-0.9):
        std_diff_temp[0,2]=std_diff_f3
    elif (label[2]<-0.3) & (label[0]>-0.5):
        std_diff_temp[0,2]=std_diff_f4
    elif (label[2]<0.5) & (label[0]>-0.3):
        std_diff_temp[0,2]=std_diff_f5
    
    test_1_total_err[i] = np.sqrt(test_1_pred_errors[i]+np.square(std_diff_temp))

In [24]:
mean_err_total = np.mean(test_1_total_err, axis=0)
print('Mean total statistical errors: \n')
for i, err in enumerate(mean_err_total):
      print(label_names[i]+':  '+"{0:.3f}".format(err)+' '+units[i])

Mean total statistical errors: 

Teff  :  67.354 K
log(g):  0.206 cgs
[Fe/H]:  0.134 dex
